In [ ]:
import Pkg

pkgs = [
"ProgressMeter",
"uCSV",
"DataFrames"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
base_directory = dirname(pwd())
metadata_file_path = joinpath(base_directory, "metadata", "ultima", "Snyder_delivery_1.xlsx - Sheet1.tsv")
metadata_table = DataFrames.DataFrame(uCSV.read(metadata_file_path, delim='\t', header=1))
sort!(metadata_table, "Customer ID")
ultima_data_dir = joinpath(base_directory, "data", "Ultima")

In [ ]:
sample_metadata = metadata_table[1, :]

In [ ]:
sample_directory = mkpath(joinpath(ultima_data_dir, sample_metadata["Customer ID"]))

In [ ]:
for remote_file in [
    sample_metadata["raw fastq"],
    sample_metadata["non-human fastq"],
    sample_metadata["kraken report all data, incl human"]
]
    local_file = joinpath(sample_directory, basename(remote_file))
    if !isfile(local_file)
        cmd = `aws s3 cp $(remote_file) $(local_file)`
        print(cmd)
        run(cmd)
    end
end

In [ ]:
remote_raw_fastq = 
local_raw_fastq = joinpath(sample_directory, basename(remote_raw_fastq))

In [ ]:
# aws s3 cp s3://bucket-name/path/to/file /local/destination

In [ ]:
# sample_metadata = metadata_table[1, :]

In [ ]:
# samples are unique by ID & pool, but same IDs are on multiple pools
# unique(metadata_table[!, ["pool nexus", "Customer ID"]])

In [ ]:
sample_metadata["Customer ID"]

In [ ]:
# conda create -n awscli -c conda-forge awscli
# pip3 install awscli --upgrade --user

In [ ]:
# for x in metadata_table[1:1, "raw fastq"]

# end

In [ ]:
# raw fastq
# non-human fastq
# kraken report all data, incl humana


In [ ]:
srr_list = "$(dirname(pwd()))/metadata/exposome/SraAccList.txt"
srr_identifiers = readlines(srr_list)

In [ ]:
srr_identifiers_to_download = filter(srr_identifier -> !isdir("$(dirname(pwd()))/data/SRA/$(srr_identifier)") || !("kraken" in readdir("$(dirname(pwd()))/data/SRA/$(srr_identifier)")), srr_identifiers)

In [ ]:
done = false
while !done
    ProgressMeter.@showprogress for srr_identifier in srr_identifiers_to_download
        sample_outdir = "$(dirname(pwd()))/data/SRA/$(srr_identifier)"
        download_and_filter_reads(outdir=sample_outdir, srr_identifier=srr_identifier)
    end
    done = true
end
@show done